# Spark - Exercício prático

Este notebook contém o exercício prático de Spark usando python. Você pode realizá-lo localmente em sua máquina (se estiver utilizando Linux) ou subir o código no Google Colab. Se estiver usando Windows, verifique o doc [2-2-spark-exercicios](2-2-spark-exercicios.md) para um tutorial de instalação do Spark no Windows.

## Configuração do ambiente. 

As próximos células fazem a instalação e configuração do java-8 e do spark em sua máquina.

In [1]:
%%bash

# Instal Java
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

## 01 - Leitura de dados

Para começar, faça a leitura do arquivo `titanic.csv` para o ambiente spark. Algumas informações importantes:

- O arquivo tem ";" como separador,
- O arquivo possui os nomes das colunas na primeira linha,
- O arquivo deve ser lido com os tipos corretos.

In [3]:
url = "https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv"

# Desenvolva seu código de leitura aqui:

from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').appName('Classify Urls').getOrCreate()
url = "https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv"
spark.sparkContext.addFile(url)
df_titanic = spark.read.csv('file:///'+SparkFiles.get('titanic.csv'), header=True, sep=';')

## 02 - Algumas análises preliminares

Exiba na tela:

1) O schema da tabela
2) Os 10 primeiros casos
3) Apenas as pessoas que sobreviveram
4) Apenas as pessoas que não sobreviveram e eram do sexo masculino
5) A média de tarifa paga para cada classe
6) A média de tarifa paga para cada classe e sexo
7) Número de sobreviventes por classe e sexo
8) Número de sobreviventes por sexo e categorias de idade (as categorias de idade devem ser construídas como: abaixo de 18 = criança, entre 18 e 40 = jovem adulto, maior de 40 = adulto maduro).

In [4]:
# 1)
# Desenvolva sua resposta aqui:
df_titanic.schema
df_titanic.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
# 2)
# Desenvolva sua resposta aqui:
df_titanic.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [6]:
# 3)
# Desenvolva sua resposta aqui:
df_titanic.filter('Survived=1').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599| 71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|    53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742| 11.1333| null|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736| 30.0708| null|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    

In [7]:
# 4)
# Desenvolva sua resposta aqui:
df_titanic.filter("Survived='0' and sex='male'").show()

+-----------+--------+------+--------------------+----+----+-----+-----+---------------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|         Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------------+-------+-----------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|      A/5 21171|   7.25|       null|       S|
|          5|       0|     3|Allen, Mr. Willia...|male|35.0|    0|    0|         373450|   8.05|       null|       S|
|          6|       0|     3|    Moran, Mr. James|male|null|    0|    0|         330877| 8.4583|       null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|male|54.0|    0|    0|          17463|51.8625|        E46|       S|
|          8|       0|     3|Palsson, Master. ...|male| 2.0|    3|    1|         349909| 21.075|       null|       S|
|         13|       0|     3|Saundercock, Mr. ...|male|2

In [8]:
# 5)
# Desenvolva sua resposta aqui:
from pyspark.sql.functions import col

# Para converter todo o dataframe:
#df_titanic.select(*(col(Fare).cast('numeric').alias(Fare) for Fare in df_titanic.columns)).printSchema()

# Para converter somente a coluna requisitada:
df_titanic = df_titanic.withColumn('Fare', col('Fare').cast('float')) # para exibir >> .printSchema()
df_titanic.groupBy('Pclass').avg('Fare').show(10)

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     3|13.675550210257411|
|     1| 84.15468752825701|
|     2| 20.66218318109927|
+------+------------------+



In [9]:
# 6)
# Desenvolva sua resposta aqui:
df_titanic.groupBy('Pclass','Sex').avg('Fare').show(10)

+------+------+------------------+
|Pclass|   Sex|         avg(Fare)|
+------+------+------------------+
|     2|female|21.970121095055028|
|     2|  male|19.741782426834106|
|     3|  male|12.661632678007186|
|     1|  male| 67.22612723366159|
|     1|female|106.12579769783832|
|     3|female|16.118809819221497|
+------+------+------------------+



In [12]:
# 7)
# Desenvolva sua resposta aqui:
df_titanic = df_titanic.withColumn('Survived', col('Survived').cast('float')) # para exibir >> .printSchema()
df_titanic.groupBy('Pclass', 'Sex').sum('Survived').show(10)

+------+------+-------------+
|Pclass|   Sex|sum(Survived)|
+------+------+-------------+
|     2|female|         70.0|
|     2|  male|         17.0|
|     3|  male|         47.0|
|     1|  male|         45.0|
|     1|female|         91.0|
|     3|female|         72.0|
+------+------+-------------+



In [11]:
# 8)
# Desenvolva sua resposta aqui:
# abaixo de 18 = criança, entre 18 e 40 = jovem adulto, maior de 40 = adulto maduro
df_titanic = df_titanic.withColumn('Age', col('Age').cast('float')) # para exibir >> .printSchema()

from pyspark.sql.functions import when
df_titanic = df_titanic.withColumn('Age', \
                      when(df_titanic['Age']<18, 'criança')\
                      .when(df_titanic['Age']==18, 'jovem adulto')\
                      .when(df_titanic['Age']<=40 , 'jovem adulto')\
                      .when(df_titanic['Age']>40 , 'adulto maduro')
                      .otherwise(df_titanic['Age']))
table_categ = df_titanic.groupBy('Sex', 'Age').count().show()

+------+-------------+-----+
|   Sex|          Age|count|
+------+-------------+-----+
|female|adulto maduro|   48|
|  male|      criança|   58|
|female|         null|   53|
|female| jovem adulto|  158|
|  male| jovem adulto|  293|
|  male|         null|  124|
|  male|adulto maduro|  102|
|female|      criança|   55|
+------+-------------+-----+



## 03 - Escreve dados

Faça uma seleção apenas das pessoas que sobreviveram e escreva os dados em um arquivo **parquet** particionado por sexo e classe.

In [32]:
# Desenvolva a resposta aqui...
from pyspark.sql import Row

df_titanic = spark.read.csv('file:///'+SparkFiles.get('titanic.csv'), header=True, sep=';')
df_titanic = df_titanic.filter('Survived=1')

titanic_parquet = df_titanic.write.partitionBy("Pclass","Sex").mode("overwrite").parquet("/Users/anacr/onboarding-ed-a3data/3-spark/titanic.parquet")

#Ex.: Consulta Pclass=1 Sex=male
titanic_parquet = spark.read.parquet("/Users/anacr/onboarding-ed-a3data/3-spark/titanic.parquet/Pclass=1/Sex=male")
titanic_parquet.show(truncate=False)

+-----------+--------+-------------------------------------------------+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Name                                             |Age |SibSp|Parch|Ticket  |Fare   |Cabin  |Embarked|
+-----------+--------+-------------------------------------------------+----+-----+-----+--------+-------+-------+--------+
|24         |1       |Sloper, Mr. William Thompson                     |28.0|0    |0    |113788  |35.5   |A6     |S       |
|56         |1       |Woolner, Mr. Hugh                                |null|0    |0    |19947   |35.5   |C52    |S       |
|98         |1       |Greenfield, Mr. William Bertram                  |23.0|0    |1    |PC 17759|63.3583|D10 D12|C       |
|188        |1       |"Romaine, Mr. Charles Hallace (""Mr C Rolmane"")"|45.0|0    |0    |111428  |26.55  |null   |S       |
|210        |1       |Blank, Mr. Henry                                 |40.0|0    |0    |112277  |31.0   |A31    |C       |
|225    